### Imports

In addition to our standard imports, we will use some that may not be as familiar to you: os, json, tqdm, math.

In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# additional imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

### Credentials and Accessing the API

You will need to use your credentials for access.

In [2]:
# Load API Credentials
with open('/Users/miran/.secret/yelp_api.json') as f:   #use your path here!
    login = json.load(f)
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)

### Define Search

To allow us to easily perform different searches in the future, we will define variables for LOCATION and TERM set for our particular search conditions. Then, when we want to use a different location or term, we can just redefine these variables. This streamlines the code and makes it more readable and reproducible.

In [3]:
# set our API call parameters 
LOCATION = 'NY,NY'
TERM = 'Pizza'

### Create a results-in-progress JSON file, but only if it doesn't exist.

This is the file where your results will be saved. Note: you must rename your JSON_FILE for different queries to prevent confusing results from other searches. We recommend you include your search terms in the filename.

In [4]:
# Specifying JSON_FILE filename (can include a folder)
# include the search terms in the filename
JSON_FILE = "Data/results_in_progress_NY_pizza.json"
JSON_FILE

'Data/results_in_progress_NY_pizza.json'

### Check if our JSON_FILE already exists. This will prevent us from accidentally overwriting an existing file.

If it doesn't exist:

Create any folders needed for the file path.
Save an empty list as JSON_File

In [5]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f'[i] {JSON_FILE} not found. Saving empty list to file.')
    
    
    # save an empty list
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
# If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/results_in_progress_NY_pizza.json not found. Saving empty list to file.


### Determine how many results are already in the file

Load the results file to determine the # of results we have previously retrieved. If you just created the file, you would expect it to be empty.

We will use this as our offset parameter for our API call. Even if this is your first API call, and the number is 0, we want to define "n_results" based on the length of "previous_results."

In [6]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 0 previous results found.


### Figure out how many pages of results we will need

We will perform our first query to get our first page of results and the total number of results. We will then (via code) calculate how many pages we will need to retrieve all of our results.

In [7]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [8]:
## How many results total?
total_results = results['total']
total_results

16000

In [9]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

### There are over 11000 businesses to retrieve from our API and we can get 20 results at a time (per "page").
We can calculate the # of results remaining by subtracting our offset (length of our previous results) from our total.
Then we can determining how many pages we will need by dividing the results by 20 (or whatever the value happens to be for results per page)
Note that we need to round up the number of pages in order to get all of the results. Even if there is only 1 result on the last page, we want to include that page! To do this we will use math.ceil.

In [10]:
# Import additional packages for controlling our loop
import time, math
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

800

### Add this page of results to .json file

Our API returns our results in JSON format, with the businesses in a list of dictionaries. We will append the first page of businesses to our previous_results (which is very likely empty) and then save to disk.

In [11]:
# join new results with old list with extend and save to file
previous_results.extend(results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)

### Set up a progress bar in our for loop.

To keep us informed about where we are in our loop, we will add a progress bar to our for loop.

TQDM is a package designed for adding animated progress bars to Python processes. It is not currently included in your dojo-env, so you are going to install it manually by opening a new Terminal/GitBash window and running the following command:

In [12]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.


### TQDM is easy to use and simply needs to know what we are looping through. If you wanted to test tqdm in action, but your loop is too fast, you can import time and use time.sleep to add a pause within your for loop. We will also use time.sleep when executing many API calls so that we do not overwhelm the server.

In [13]:
from tqdm.notebook import tqdm_notebook
import time
for i in tqdm_notebook(range(n_pages)):
    # adds 200 ms pause
    time.sleep(.2) 

  0%|          | 0/800 [00:00<?, ?it/s]

### For Loop to call each page
The loop below will iterate through each page of the results by starting at the appropriate offset. It will then append the results to the previous_results. This may take some time, so check out the progress bar!

In [14]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    # add a 200ms pause
    time.sleep(.2)

  0%|          | 0/800 [00:00<?, ?it/s]

YelpAPIError: VALIDATION_ERROR: Too many results requested, limit+offset must be <= 1000.

### The Programmatic Way
Just like we used the os module create new folders, we can also use it to delete (or remove) files.

We simply use the os.remove function and pass in our filename to delete.

We can then use our os.path.isfile function again to confirm the file no longer exists.

In [15]:
## delete file and confirm it no longer exits.
os.remove(JSON_FILE)
os.path.isfile(JSON_FILE)

False

###Now that we've deleted the file, we need to re-run our code to create it.

This process is begging to be turned into a function so we can we easily repeat it.

While we are making it a function, why don't we go ahead and add the option to delete the JSON file it if already exists, just like we did above.

So let's make a create_json_file function that accepts the JSON_FILE filename as first argument and a second argument called delete_if_exists and set to it to False by default.

This way, it will not automatically delete previous search results. We will have to explicitly say delete_if_exists = True to do so.

In [17]:
def create_json_file(JSON_FILE,  delete_if_exists=False):
    
    ## Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)
    
    ## If it DOES exist:
    if file_exists == True:
        
        ## Check if user wants to delete if exists
        if delete_if_exists==True:
            
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            ## delete file and confirm it no longer exits.
            os.remove(JSON_FILE)
            ## Recursive call to function after old file deleted
            create_json_file(JSON_FILE,delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE} already exists.")            
            
            
    ## If it does NOT exist:
    else:
        
        ## INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        
        ## CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)
        
        ## If JSON_FILE included a folder:
        if len(folder)>0:
            # create the folder
            os.makedirs(folder,exist_ok=True)
        ## Save empty list to start the json file
        with open(JSON_FILE,'w') as f:
            json.dump([],f)  

Now that we have our new function, we can use it with delete_if_exists=True to delete our previous results and start over. We will also need to repeat the steps to recreate our n-results, total_results, results_per_page, and n_pages variables that we created before our first attempted loop.

In [18]:
## Create a new empty json file (exist the previous if it exists)
create_json_file(JSON_FILE, delete_if_exists=True)
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
## How many results total?
total_results = results['total']
## How many did we get the details for?
results_per_page = len(results['businesses'])
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

[i] Data/results_in_progress_NY_pizza.json not found. Saving empty list to new file.
- 0 previous results found.


810

In [19]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)

  0%|          | 0/810 [00:00<?, ?it/s]

Exceeded 1000 api calls. Stopping loop.


### After the loop has finished

Convert .json to dataframe
Load in the "results in progress" JSON file into a DataFrame:

In [20]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,ysqgdbSrezXgVwER2kQWKA,julianas-brooklyn-3,Juliana's,https://s3-media2.fl.yelpcdn.com/bphoto/NVoLFl...,False,https://www.yelp.com/biz/julianas-brooklyn-3?a...,2634,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,"{'latitude': 40.70274718768062, 'longitude': -...",[delivery],$$,"{'address1': '19 Old Fulton St', 'address2': '...",+17185966700,(718) 596-6700,1289.857286
1,zj8Lq1T8KIC5zwFief15jg,prince-street-pizza-new-york-2,Prince Street Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/PfI8oV...,False,https://www.yelp.com/biz/prince-street-pizza-n...,4863,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 40.72308755605564, 'longitude': -...","[pickup, delivery]",$,"{'address1': '27 Prince St', 'address2': None,...",+12129664100,(212) 966-4100,2209.311618
2,WG639VkTjmK5dzydd1BBJA,rubirosa-new-york-2,Rubirosa,https://s3-media3.fl.yelpcdn.com/bphoto/F65qqO...,False,https://www.yelp.com/biz/rubirosa-new-york-2?a...,2985,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.5,"{'latitude': 40.722766, 'longitude': -73.996233}",[pickup],$$,"{'address1': '235 Mulberry St', 'address2': ''...",+12129650500,(212) 965-0500,2268.491950
3,v1DHGRNCH9247WLYoaoA9A,l-industrie-pizzeria-brooklyn,L'industrie Pizzeria,https://s3-media3.fl.yelpcdn.com/bphoto/KKB2Tr...,False,https://www.yelp.com/biz/l-industrie-pizzeria-...,934,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,"{'latitude': 40.71162, 'longitude': -73.95783}",[delivery],$,"{'address1': '254 S 2nd St', 'address2': '', '...",+17185990002,(718) 599-0002,1868.295512
4,WIhm0W9197f_rRtDziq5qQ,lombardis-pizza-new-york-4,Lombardi's Pizza,https://s3-media2.fl.yelpcdn.com/bphoto/m_9TT6...,False,https://www.yelp.com/biz/lombardis-pizza-new-y...,6562,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 40.7215934960083, 'longitude': -7...","[pickup, delivery]",$$,"{'address1': '32 Spring St', 'address2': '', '...",+12129417994,(212) 941-7994,2133.973859


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
995,epDvy_G3YITZRbhqcExkBg,hops-hill-brooklyn,Hops Hill,https://s3-media3.fl.yelpcdn.com/bphoto/AaoRaS...,False,https://www.yelp.com/biz/hops-hill-brooklyn?ad...,68,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...",4.5,"{'latitude': 40.68309, 'longitude': -73.96557}","[delivery, pickup]",$$,"{'address1': '886 Fulton St', 'address2': '', ...",+13479874334,(347) 987-4334,2897.313477
996,Uuwc9go0iaHRb8fwVjY_cA,franks-pizzeria-north-bergen,Frank's Pizzeria,https://s3-media3.fl.yelpcdn.com/bphoto/cKI6xY...,False,https://www.yelp.com/biz/franks-pizzeria-north...,35,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",3.5,"{'latitude': 40.783573, 'longitude': -74.033065}","[delivery, pickup]",$,"{'address1': '4410 Bergen Tpke', 'address2': N...",+12018663267,(201) 866-3267,9650.931568
997,LMwwufXN6Ys68GCekGJz-g,vesta-wood-fired-east-rutherford-2,Vesta Wood Fired,https://s3-media3.fl.yelpcdn.com/bphoto/tHjd3c...,False,https://www.yelp.com/biz/vesta-wood-fired-east...,540,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 40.83406, 'longitude': -74.09035}","[delivery, pickup]",$$,"{'address1': '64 Hoboken Rd', 'address2': '', ...",+12019396012,(201) 939-6012,16939.435686
998,4aF0F8w7yXX9o5_QFky_ig,celestine-brooklyn,Celestine,https://s3-media3.fl.yelpcdn.com/bphoto/p98BTW...,False,https://www.yelp.com/biz/celestine-brooklyn?ad...,282,"[{'alias': 'mediterranean', 'title': 'Mediterr...",4.0,"{'latitude': 40.704676, 'longitude': -73.987975}",[delivery],$$$,"{'address1': '1 John St', 'address2': '', 'add...",+17185225356,(718) 522-5356,783.735553
999,z5CgYMZeFmJYeWV3CyVhmA,dnl-pizza-plus-jamaica,DNL pizza plus,https://s3-media3.fl.yelpcdn.com/bphoto/D7HgkV...,False,https://www.yelp.com/biz/dnl-pizza-plus-jamaic...,18,"[{'alias': 'pizza', 'title': 'Pizza'}]",3.0,"{'latitude': 40.672853, 'longitude': -73.788645}","[delivery, pickup]",$$,"{'address1': '149-21 Rockaway Blvd', 'address2...",+17185292400,(718) 529-2400,16507.267402


### Check for duplicates
Check for and remove any duplicate results.

In [21]:
# check for duplicate results
final_df.duplicated().sum()

TypeError: unhashable type: 'list'

Because our yelp results include columns that contain lists, we cannot check every column in dataframe for duplicates.

Instead, we can use the subset argument for df.duplicated() and df.drop_duplicates() to only check the id column for duplicates.

In [22]:
# check for duplicate ID's 
final_df.duplicated(subset='id').sum()

216

In [23]:
## Drop duplicate ids and confirm there are no more duplicates
final_df = final_df.drop_duplicates(subset='id')
final_df.duplicated(subset='id').sum()

0

### Save the final DataFrame to a .csv (or a .csv.gz if its too big for the GitHub file size limit).

In [24]:
# save the final results to a compressed csv
final_df.to_csv('Data/final_results_NY_pizza.csv.gz', compression='gzip',index=False)